In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

def preprocess_data():
    """Preprocess the collected data for model training"""
    label_map = {label:num for num, label in enumerate(actions)}
    
    sequences, labels = [], []
    for action in actions:
        for sequence in range(no_sequences):
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])
    
    X = np.array(sequences)
    y = to_categorical(labels).astype(int)
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

def create_tensorflow_dataset(X_train, y_train, X_test, y_test):
    """Create TensorFlow datasets for training"""
    import tensorflow as tf
    
    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    
    # Shuffle and batch the datasets
    BATCH_SIZE = 16
    train_dataset = train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
    return train_dataset, test_dataset

def augment_sequence(sequence):
    """Apply random augmentations to a sequence"""
    # Small random noise
    noise = np.random.normal(0, 0.01, sequence.shape)
    augmented = sequence + noise
    
    # Random temporal warping (speed variation)
    if np.random.rand() > 0.5:
        orig_len = sequence.shape[0]
        new_len = int(orig_len * np.random.uniform(0.8, 1.2))
        augmented = np.array([np.interp(
            np.linspace(0, orig_len-1, new_len),
            np.arange(orig_len),
            augmented[:,i]
        ) for i in range(augmented.shape[1])]).T
    
    return augmented

In [5]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (LSTM, Dense, Dropout, Masking, 
                                    BatchNormalization, Bidirectional, 
                                    Input, Flatten)
from tensorflow.keras.callbacks import (TensorBoard, ModelCheckpoint, 
                                      EarlyStopping, ReduceLROnPlateau)

# Configuration (must match data collection)
ACTIONS = ['hello', 'my', 'name', 'is','i', 'study', 'in', 'department','am', ] # Must match exactly
NO_SEQUENCES = 50  # Must match
SEQUENCE_LENGTH = 40  # Must match
DATA_PATH = 'sign_language_data'

def preprocess_data():
    """Preprocess the collected data for model training"""
    label_map = {label:num for num, label in enumerate(ACTIONS)}
    
    sequences, labels = [], []
    for action in ACTIONS:
        for sequence in range(NO_SEQUENCES):
            window = []
            valid_frames = 0
            seq_path = os.path.join(DATA_PATH, action, str(sequence))
            
            # Skip if sequence directory doesn't exist
            if not os.path.exists(seq_path):
                print(f"Warning: Missing sequence - {action}/{sequence}")
                continue
                
            for frame_num in range(SEQUENCE_LENGTH):
                frame_path = os.path.join(seq_path, f"{frame_num}.npy")
                if os.path.exists(frame_path):
                    res = np.load(frame_path)
                    window.append(res)
                    valid_frames += 1
                else:
                    window.append(np.zeros(1662))  # Pad with zeros
            
            # Only add sequences with sufficient data
            if valid_frames >= SEQUENCE_LENGTH // 2:
                sequences.append(window)
                labels.append(label_map[action])
            else:
                print(f"Warning: Insufficient frames ({valid_frames}) in {action}/{sequence}")
    
    if not sequences:
        raise ValueError("No valid training data found!")
    
    X = np.array(sequences)
    y = to_categorical(labels).astype(int)
    
    # Check if we have enough samples for splitting
    if len(sequences) < 2:
        raise ValueError(f"Insufficient data samples ({len(sequences)}). Need at least 2 for train-test split.")
    
    return train_test_split(X, y, test_size=0.2, random_state=42)

def build_model():
    """Build LSTM model without attention for simpler implementation"""
    model = Sequential([
        Masking(mask_value=0., input_shape=(SEQUENCE_LENGTH, 1662)),
        
        # First bidirectional LSTM layer
        Bidirectional(LSTM(256, return_sequences=True)),
        BatchNormalization(),
        Dropout(0.3),
        
        # Second bidirectional LSTM layer
        Bidirectional(LSTM(128, return_sequences=True)),
        BatchNormalization(),
        Dropout(0.3),
        
        # Final LSTM layer
        LSTM(64),
        BatchNormalization(),
        Dropout(0.3),
        
        # Dense layers
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(64, activation='relu'),
        Dense(len(ACTIONS), activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy', 
                tf.keras.metrics.TopKCategoricalAccuracy(k=2, name='top2_accuracy')]
    )
    return model

def train_model(X_train, y_train, X_test, y_test):
    os.makedirs('Logs', exist_ok=True)
    
    callbacks = [
        ModelCheckpoint(
            'best_model.keras',
            monitor='val_top2_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    model = build_model()
    
    # Calculate class weights to handle imbalance
    class_counts = np.sum(y_train, axis=0)
    class_weights = {i: 1./count for i, count in enumerate(class_counts)}
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=200,
        batch_size=32,
        class_weight=class_weights,
        callbacks=callbacks
    )
    return model, history

def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy:.4f}')
    model.save('sign_language_model.keras')

if __name__ == "__main__":
    print("Preprocessing data...")
    try:
        X_train, X_test, y_train, y_test = preprocess_data()
        print(f"Data shapes - X_train: {X_train.shape}, y_train: {y_train.shape}")
        
        print("\nTraining model...")
        model, history = train_model(X_train, y_train, X_test, y_test)
        
        print("\nEvaluating model...")
        evaluate_model(model, X_test, y_test)
    except Exception as e:
        print(f"Error: {e}")
        print("Please ensure you have collected data for all actions")

Preprocessing data...
Data shapes - X_train: (360, 40, 1662), y_train: (360, 9)

Training model...
Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.1221 - loss: 0.9181 - top2_accuracy: 0.2678
Epoch 1: val_top2_accuracy improved from -inf to 0.22222, saving model to best_model.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 362ms/step - accuracy: 0.1242 - loss: 0.9166 - top2_accuracy: 0.2707 - val_accuracy: 0.1333 - val_loss: 2.9926 - val_top2_accuracy: 0.2222 - learning_rate: 5.0000e-04
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.2526 - loss: 0.8264 - top2_accuracy: 0.3956
Epoch 2: val_top2_accuracy did not improve from 0.22222
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 268ms/step - accuracy: 0.2534 - loss: 0.8252 - top2_accuracy: 0.3968 - val_accuracy: 0.1333 - val_loss: 2.9097 - val_top2_accuracy: 0.2000 - learning_rate: 5.0000e-04
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 0.3486 - loss: 0.7489 - top2_accuracy: 0.5217
Epoch 3: val_top2_accuracy